# Reading data into Astropy Tables

## Objectives

 - Read ASCII files with a defined format
 - Learn basic operations with `astropy.tables`
 - Ingest header information
 - VOTables

## Reading data

Our first task with python was to read a `csv` file using `np.loadtxt()`.
That function has few properties to define the dlimiter of the columns, skip rows, read commented lines, convert values while reading, etc.

However, the result is an array, without the information of the metadata that file may have included (name, units, ...).

Astropy offers a ascii reader that improves many of these steps while provides templates to read common ascii files in astronomy.


In [1]:
from astropy.io import ascii

In [2]:
# Read a sample file: sources.dat
data = ascii.read("sources.dat")
data

obsid,redshift,X,Y,object
int64,float64,int64,int64,str11
3102,0.32,4167,4085,Q1250+568-A
877,0.22,4378,3892,Source 82


Automatically, read has identified the header and the format of each column. The result is a `Table` object, and that brings some additional properties.

In [3]:
# Show the info of the data read
data.info

/home/dvd/.conda/envs/swc/lib/python2.7/site-packages/astropy/table/column.py:268: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return self.data.__eq__(other)


<Table length=2>
  name    dtype 
-------- -------
   obsid   int64
redshift float64
       X   int64
       Y   int64
  object   str11

In [4]:
# Get the name of the columns
data.colnames

['obsid', 'redshift', 'X', 'Y', 'object']

In [5]:
# Get just the values of a particular column
data['obsid']

3102
877


In [6]:
# get the first element
data['obsid', 'redshift'][0]

obsid,redshift
int64,float64
3102,0.32


Astropy [can read a variety of formats](http://astropy.readthedocs.org/en/stable/io/ascii/index.html#supported-formats) easily.
The following example uses a quite 

In [7]:
# Read the data from the source
table = ascii.read("ftp://cdsarc.u-strasbg.fr/pub/cats/VII/253/snrs.dat",
                   readme="ftp://cdsarc.u-strasbg.fr/pub/cats/VII/253/ReadMe")

In [8]:
# See the stats of the table
table.info('stats')

<Table masked=True length=274>
   name         mean           std       min  max   n_bad
---------- -------------- -------------- --- ------ -----
       SNR             --             --  --     --     0
       RAh  16.0547445255  4.15229196762   0     23     0
       RAm  28.8576642336  16.9123382708   0     59     0
       RAs   28.102189781  18.5923556505   0     59     0
       DE-             --             --  --     --     0
       DEd   33.602189781  19.4333634671   0     72     0
       DEm  29.6459854015  17.7768672558   0     59     0
   MajDiam  30.9124087591  42.1254815567 1.5  310.0     0
       ---             --             --  --     --   164
   MinDiam  23.4909090909  33.3816758266 2.0  240.0   164
 u_MinDiam             --             --  --     --   243
      type             --             --  --     --     0
 l_S(1GHz)             --             --  --     --   270
   S(1GHz)  42.6488549618  212.136906631 0.3 2720.0    12
 u_S(1GHz)             --             -- 

/home/dvd/.conda/envs/swc/lib/python2.7/site-packages/astropy/table/info.py:94: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if np.all(info[name] == ''):


In [9]:
# If we want to see the first 10 entries
table[0:10]

SNR,RAh,RAm,RAs,DE-,DEd,DEm,MajDiam,---,MinDiam,u_MinDiam,type,l_S(1GHz),S(1GHz),u_S(1GHz),Sp-Index,u_Sp-Index,Names
,h,min,s,,deg,arcmin,arcmin,,arcmin,,,,Jy,,,,
str11,int64,int64,int64,str1,int64,int64,float64,str1,float64,str1,str2,str1,float64,str1,float64,str1,str26
G000.0+00.0,17,45,44,-,29,0,3.5,x,2.5,--,S,--,100.0,?,0.8,?,Sgr A East
G000.3+00.0,17,46,15,-,28,38,15.0,x,8.0,--,S,--,22.0,--,0.6,--,--
G000.9+00.1,17,47,21,-,28,9,8.0,--,--,--,C,--,18.0,?,--,v,--
G001.0-00.1,17,48,30,-,28,9,8.0,--,--,--,S,--,15.0,--,0.6,?,--
G001.4-00.1,17,49,39,-,27,46,10.0,--,--,--,S,--,2.0,?,--,?,--
G001.9+00.3,17,48,45,-,27,10,1.5,--,--,--,S,--,0.6,--,0.6,--,--
G003.7-00.2,17,55,26,-,25,50,14.0,x,11.0,--,S,--,2.3,--,0.65,--,--
G003.8+00.3,17,52,55,-,25,28,18.0,--,--,--,S?,--,3.0,?,0.6,--,--


In [10]:
# the units are also stored, we can extract them too
table['MajDiam'].quantity.to('rad')[0:3]

<Quantity [ 0.00101811, 0.00436332, 0.00232711] rad>

In [11]:
# Adding values of different columns
(table['RAh'] + table['RAm'] + table['RAs'])[0:3]


106
78
85


In [12]:
# adding values of different columns but being aware of column units
(table['RAh'].quantity + table['RAm'].quantity + table['RAs'].quantity)[0:3]

<Quantity [ 17.76222222, 17.77083333, 17.78916667] h>

In [13]:
# Create a new column in the table
table['RA'] = table['RAh'].quantity + table['RAm'].quantity + table['RAs'].quantity

In [14]:
# Show table's new column 
table['RA'][0:3]

17.7622222222
17.7708333333
17.7891666667


In [15]:
# add a description to the new column
table['RA'].description = table['RAh'].description

In [16]:
# Now it does show the values
table['RA'][0:3]


17.7622222222
17.7708333333
17.7891666667


In [17]:
# Using numpy to calculate the sin of the RA
import numpy as np
np.sin(table['RA'].quantity)

TypeError: Can only apply 'sin' function to quantities with angle units

In [18]:
# Let's change the units...
import astropy.units as u
table['RA'].unit = u.hourangle

In [19]:
# does the sin now works?
np.sin(table['RA'].quantity)

<Quantity [-0.99806309,-0.9982008 ,-0.99847709,-0.99874134,-0.99898044,
           -0.99879546,-0.99980149,-0.99952242,-0.99924325,-0.99183891,
           -0.9932805 ,-0.99946459,-0.99995531,-0.99991809,-0.99847307,
           -0.99993971,-0.99975403,-0.99999762,-0.99790622,-0.99995462,
           -0.999968  ,-0.99998813,-0.9971138 ,-0.99980149,-0.99444562,
           -0.99971206,-0.99985022,-0.99948345,-0.99974917,-0.99891373,
           -0.99920603,-0.99903549,-0.99812146,-0.99844887,-0.99908901,
           -0.99875226,-0.9988933 ,-0.99853273,-0.99858735,-0.99878831,
           -0.99857573,-0.99831238,-0.99823551,-0.99644401,-0.99807213,
           -0.99760446,-0.99757419,-0.9945219 ,-0.99690591,-0.99661351,
           -0.99542404,-0.99163366,-0.99512753,-0.9958315 ,-0.99389695,
           -0.99541708,-0.99091487,-0.99491006,-0.98975548,-0.9942604 ,
           -0.99361295,-0.99453709,-0.99153952,-0.99408796,-0.99248398,
           -0.9926257 ,-0.99074784,-0.98930418,-0.9909637 ,-0.98

## Properties when reading

the reading of the table has many properties, let's imagine the following easy example:

In [20]:
weather_data = """
# Country = Finland
# City = Helsinki
# Longitud = 24.9375
# Latitud = 60.170833
# Week = 32
# Year = 2015
day, precip, type
Mon,1.5,rain
Tues,,
Wed,1.1,snow
Thur,2.3,rain
Fri,0.2,
Sat,1.1,snow
Sun,5.4,snow
"""

In [21]:
# Read the table
weather = ascii.read(weather_data)

In [22]:
# Blank values are interpreted by default as bad/missing values
weather.info('stats')

<Table masked=True length=7>
 name       mean          std      min max n_bad
------ ------------- ------------- --- --- -----
   day            --            --  --  --     0
precip 1.93333333333 1.66999667332 0.2 5.4     1
  type            --            --  --  --     2


In [23]:
# Let's define missing values for the columns we want:
weather['type'].fill_value = 'N/A'
weather['precip'].fill_value = -999

In [24]:
# Use filled to show the value filled.
weather.filled()

day,precip,type
str4,float64,str4
Mon,1.5,rain
Tues,-999.0,N/A
Wed,1.1,snow
Thur,2.3,rain
Fri,0.2,N/A
Sat,1.1,snow
Sun,5.4,snow


In [25]:
# We can see the meta as a dictionary, but not as key, value pairs
weather.meta

OrderedDict([('comments',
              ['Country = Finland',
               'City = Helsinki',
               'Longitud = 24.9375',
               'Latitud = 60.170833',
               'Week = 32',
               'Year = 2015'])])

In [26]:
# To get it the header as a table
header = ascii.read(weather.meta['comments'], delimiter='=',
                    format='no_header', names=['key', 'val'])
print(header)

  key       val   
-------- ---------
 Country   Finland
    City  Helsinki
Longitud   24.9375
 Latitud 60.170833
    Week        32
    Year      2015


When the values are not empty, then the keyword `fill_values` on `read` has [to be used](http://astropy.readthedocs.org/en/stable/io/ascii/read.html#bad-or-missing-values).


## Reading VOTables

VOTables are an special type of tables which should be self-consistent and can be tied to a particular scheme.
This mean the file will contain where the data comes from (and which query produced it) and the properties for each field, making it easier to ingest by a machine.

In [27]:
from astropy.io.votable import parse_single_table

In [28]:
# Read the example table from HELIO (hfc_ar.xml)
table = parse_single_table("hfc_ar.xml")

In [29]:
# See the fields of the table
table.fields

[<FIELD ID="ID_AR" datatype="int" name="ID_AR" ucd="meta.id" utype="ar.id"/>,
 <FIELD ID="DATE_OBS" arraysize="*" datatype="char" name="DATE_OBS" ucd="time.start;obs" utype="image.time_obs | time_period.time_start.first_observation_time"/>,
 <FIELD ID="NOAA_NUMBER" datatype="int" name="NOAA_NUMBER" ucd="meta.id" utype="ar.noaa_solar_region_id"/>,
 <FIELD ID="FEAT_HG_LAT_DEG" datatype="float" name="FEAT_HG_LAT_DEG" ucd="pos.heliographic;pos.barycenter;pos.bodyrc.lat" unit="deg" utype="feature.centre.lat_hg"/>,
 <FIELD ID="FEAT_AREA_DEG2" datatype="float" name="FEAT_AREA_DEG2" ucd="phys.area" unit="deg2" utype="feature.area.area_deg_sq"/>,
 <FIELD ID="FEAT_X_ARCSEC" datatype="double" name="FEAT_X_ARCSEC" ucd="pos.wcs;pos.barycenter;Qualifier.DirectionAngle.AzimuthAngle" unit="arcs" utype="feature.centre.x_cart"/>,
 <FIELD ID="FEAT_Y_ARCSEC" datatype="double" name="FEAT_Y_ARCSEC" ucd="pos.wcs;pos.barycenter;Qualifier.DirectionAngle.ElevationAngle" unit="arcs" utype="feature.centre.y_cart"

In [30]:
# extract one  (NOAA_NUMBER) or all of the columns
NOAA = table.array['NOAA_NUMBER']

In [31]:
# Show the data
NOAA.data

array([      10321, -2147483648,       10325, ..., -2147483648,
             10332, -2147483648], dtype=int32)

In [32]:
# See the mask
NOAA.mask

array([False,  True, False, ...,  True, False,  True], dtype=bool)

In [33]:
# Shee the whole array.
NOAA

masked_array(data = [10321 -- 10325 ..., -- 10332 --],
             mask = [False  True False ...,  True False  True],
       fill_value = 999999)

In [34]:
# Convert the table to an astropy table
asttable = table.to_table()

In [35]:
# See the table
asttable

ID_AR,DATE_OBS,NOAA_NUMBER,FEAT_HG_LAT_DEG,FEAT_AREA_DEG2,FEAT_X_ARCSEC,FEAT_Y_ARCSEC,FEAT_MAX_INT,FEAT_MIN_INT,FEAT_MEAN_INT
,,,deg,deg2,arcs,arcs,gauss,gauss,gauss
int32,object,int32,float32,float32,float64,float64,float32,float32,float32
247523,2003-04-01T00:00:00,10321,4.87889,190.33,273.78399999999999,185.994,1789.54,-2573.3201,-0.46051401
247528,2003-04-01T00:00:00,--,-12.1797,160.16,-188.178,-96.1541,2147.6201,-1722.37,-46.435699
247539,2003-04-01T00:00:00,10325,12.3932,147.224,-458.50900000000001,298.358,3202.8701,-1377.66,-2.20612
247545,2003-04-01T00:00:00,--,-15.9363,129.17799,-577.27200000000005,-179.56999999999999,766.06,-2342.4299,32.803101
247549,2003-04-01T00:00:00,10323,-7.57478,108.752,472.27600000000001,-31.2653,2495.7,-2757.1101,-4.3433499
247563,2003-04-01T00:00:00,10318,-14.9789,77.475998,705.50699999999995,-177.726,1379.92,-1706.51,-18.2155
247581,2003-04-01T00:00:00,10319,12.0031,62.118,799.56500000000005,254.83600000000001,2780.72,-1257.1,-11.4169
247591,2003-04-01T00:00:00,--,9.3859196,46.102001,-251.09299999999999,260.08199999999999,902.16302,0.0,51.014099


In [36]:
# Different results because quantities are not 
print(np.sin(asttable['FEAT_HG_LAT_DEG'][0:5]))
print(np.sin(asttable['FEAT_HG_LAT_DEG'][0:5].quantity))


FEAT_HG_LAT_DEG
---------------
      -0.986171
       0.377107
      -0.172306
       0.226358
      -0.961276
[ 0.08504982 -0.21097848  0.21461941 -0.27456847 -0.13182007]


In [37]:
# And it can also be converted to other units
print(asttable[0:5]['FEAT_AREA_DEG2'].quantity.to('arcmin2'))

[ 685188.       576576.       530006.375    465040.78125  391507.1875 ] arcmin2
